In [40]:
import cv2
import json
import numpy as np
import pandas as pd
import random
import os
import shutil
from collections import Counter
from PIL import Image
import imageio
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.image as mpimg
from ultralytics import YOLO
from keras.layers import Flatten, Dense
from keras.models import Model
from keras.preprocessing import image
import tensorflow as tf
from tensorflow import keras

In [50]:
import json
import os

def json_to_yolo(data):
    yolo_format = []

    image_width = data["description"]["imageWidth"]
    image_height = data["description"]["imageHeight"]

    for annotation in data["annotations"].get("environment", []):
        # For polygon shapes in environment, currently skipping as YOLO uses bounding boxes
        pass

    for annotation in data["annotations"].get("PM", []):
        if annotation["shape_type"] == "bbox":
            x_center = (annotation["points"][0] + annotation["points"][2]) / 2 / image_width
            y_center = (annotation["points"][1] + annotation["points"][3]) / 2 / image_height
            width = (annotation["points"][2] - annotation["points"][0]) / image_width
            height = (annotation["points"][3] - annotation["points"][1]) / image_height

            yolo_format.append(f"{annotation['PM_code']} {x_center} {y_center} {width} {height}")

    return yolo_format


def convert_to_utf8(filepath):
    # 파일을 바이너리 모드로 읽습니다.
    with open(filepath, 'rb') as file:
        content = file.read()

    # utf-8로 다시 씁니다.
    with open(filepath, 'w', encoding='utf-8') as file:
        file.write(content.decode('utf-8', 'ignore'))


def load_json_from_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return json.load(f)


def save_yolo_to_txt(yolo_data, filepath, json_filename):
    txt_filename = os.path.splitext(json_filename)[0] + ".txt"
    txt_filepath = os.path.join(filepath, txt_filename)
    with open(txt_filepath, 'w', encoding='utf-8') as f:
        for line in yolo_data:
            f.write(line + '\n')


## 예시
# json_file_path = '/Users/bumsoojoe/Desktop/팀프로젝트2_교통법규/database/toy_data/json/C000003_037_0234_C_N_F_0.json'
# txt_directory_path = '/Users/bumsoojoe/Desktop/팀프로젝트2_교통법규/database/toy_data/txt'
# json_filename = os.path.basename(json_file_path)
# data = load_json_from_file(json_file_path)
# yolo_annotations = json_to_yolo(data)
# save_yolo_to_txt(yolo_annotations, txt_directory_path, json_filename)

proof_label_path = '/Users/bumsoojoe/Desktop/팀프로젝트2_교통법규/Database/2.Validation/proof_label'
direction_label_path = '/Users/bumsoojoe/Desktop/팀프로젝트2_교통법규/Database/2.Validation/direction_label'
txt_directory_path = '/Users/bumsoojoe/Desktop/팀프로젝트2_교통법규/Database/2.Validation/label_EDA'

def json_to_txt(label_path,txt_directory_path):
    all_data_list = []
    failed_files = []  # 에러가 발생한 파일 이름을 저장할 리스트
    filenames = list(os.listdir(label_path))
    for i in filenames:
        json_file_path = str(proof_label_path + '/' + i)
        convert_to_utf8(json_file_path)
        json_filename = os.path.basename(json_file_path)
        
        try:
            data = load_json_from_file(json_file_path)
            yolo_annotations = json_to_yolo(data)
            save_yolo_to_txt(yolo_annotations, txt_directory_path, json_filename)

            for ann in yolo_annotations:
                ann = i + ' 0 ' + ann
                all_data_list.append(ann.split(' '))
        
        except json.JSONDecodeError:
            failed_files.append(i)
    print("Failed files:", failed_files)
    return all_data_list

filenames2 = list(os.listdir(direction_label_path))
for i in filenames2:
    json_file_path = str(direction_label_path + '/' + i)
    convert_to_utf8(json_file_path)
    json_filename = os.path.basename(json_file_path)

    try:
        data = load_json_from_file(json_file_path)
        yolo_annotations = json_to_yolo(data)
        save_yolo_to_txt(yolo_annotations, txt_directory_path, json_filename)

        for ann in yolo_annotations:
            ann = i + ' 1 ' + ann
            all_data_list.append(ann.split(' '))
    
    except json.JSONDecodeError:
        failed_files.append(i)

df = pd.DataFrame(all_data_list, columns=['name', 'proof', 'class', 'x_center', 'y_center', 'width', 'height'])
print("Failed files:", failed_files)  # 실패한 파일 이름 출력
df


Failed files: ['.DS_Store', '.DS_Store']


,name,proof,class,x_center,y_center,width,height
0,C000052_048_0294_C_D_R_0.json,0,13,0.34322916666666664,0.1912037037037037,-0.5583333333333333,-0.1824074074074074
1,C000052_048_0294_C_D_R_0.json,0,17,0.18046875,0.15,-0.2921875,-0.14814814814814814
2,C000438_015_0162_C_D_F_0.json,0,13,0.21817859457365157,0.12288584780163853,-0.39039069495452605,-0.12094419641328766
3,C000438_015_0162_C_D_F_0.json,0,13,0.2082696786393081,0.13215784931130975,-0.3667269690681327,-0.1311792666888792
4,C000438_015_0162_C_D_F_0.json,0,22,0.09494389759681078,0.19458648602970438,-0.13671905603897463,-0.1906570462733129
...,...,...,...,...,...,...,...
97164,F000064_000_7964_C_D_F_1.json,1,14,0.2446154517351854,0.15661482447574823,-0.46381031876685735,-0.20647394914493297
97165,T015369_076_0126_C_D_F_1.json,1,28,0.06213209973849883,0.21646330182238982,-0.07607584028068735,-0.20931777250375974
97166,T015369_076_0126_C_D_F_1.json,1,28,0.08992072020837111,0.24783698986513497,-0.1130202489978586,-0.2241489704876029
97167,T015369_076_0126_C_D_F_1.json,1,28,0.16268896311983191,0.3430556263963025,-0.22760444381236739,-0.22604649928041354


In [13]:
yolo_annotations

['17 0.3679995369146758 0.1218442288592682 -0.695588613187432 -0.12387553593485646',
 '18 0.35130208333333335 0.1800925925925926 -0.6369791666666667 -0.14735577090395538',
 '18 0.3681843996153487 0.2235222980417327 -0.661547867435969 -0.19369614465727547']

In [38]:
proof_label_path = '/Users/bumsoojoe/Desktop/팀프로젝트2_교통법규/Database/2.Validation/proof_label'
direction_label_path = '/Users/bumsoojoe/Desktop/팀프로젝트2_교통법규/Database/2.Validation/direction_label'
txt_directory_path = '/Users/bumsoojoe/Desktop/팀프로젝트2_교통법규/Database/2.Validation/label_EDA'

filenames = list(os.listdir(proof_label_path))
filenames

['C000052_048_0294_C_D_R_0.json',
 'C000438_015_0162_C_D_F_0.json',
 'S000569_054_0315_C_N_F_0.json',
 'M100653_007_0080_C_D_F_0.json',
 'S000570_121_0054_C_D_F_0.json',
 'S000514_010_0024_C_N_F_0.json',
 'S000885_009_0207_C_D_R_0.json',
 'S000634_134_0060_C_D_F_0.json',
 'S000635_053_0264_C_D_F_0.json',
 'C000592_012_0072_C_D_F_0.json',
 'S000289_001_0252_C_D_F_0.json',
 'S000568_045_0063_C_N_F_0.json',
 'C001162_016_0104_C_D_F_0.json',
 'C000662_014_0304_C_D_F_0.json',
 'S001763_008_0018_C_N_F_0.json',
 'S001242_025_0100_C_D_F_0.json',
 'C000420_066_0180_C_D_F_0.json',
 'C000003_045_0050_C_N_F_0.json',
 'S000714_012_0266_C_D_F_0.json',
 'C000438_001_0280_C_D_F_0.json',
 'S000040_003_0036_C_D_R_0.json',
 'C000037_009_0060_C_D_F_0.json',
 'C000055_016_0021_C_D_R_0.json',
 'C000052_097_0252_C_D_R_0.json',
 'S000398_005_0154_C_D_F_0.json',
 'S001461_023_0270_C_N_F_0.json',
 'S000567_042_0216_C_N_F_0.json',
 'S000056_000_0200_C_D_F_0.json',
 'C000482_044_0014_C_D_F_0.json',
 'S000757_001_

In [19]:
all_data_list = []

for ann in yolo_annotations:
    ann = '' + ann
    list_ass.append(ann.split(' '))
list_ass

[['C000003_037_0234_C_N_F_0',
  '17',
  '0.3679995369146758',
  '0.1218442288592682',
  '-0.695588613187432',
  '-0.12387553593485646'],
 ['C000003_037_0234_C_N_F_0',
  '18',
  '0.35130208333333335',
  '0.1800925925925926',
  '-0.6369791666666667',
  '-0.14735577090395538'],
 ['C000003_037_0234_C_N_F_0',
  '18',
  '0.3681843996153487',
  '0.2235222980417327',
  '-0.661547867435969',
  '-0.19369614465727547']]

In [21]:
df = pd.DataFrame(list_ass,columns=['name', 'class','x_center','y_center','width','height'])
df

,name,class,x_center,y_center,width,height
0,C000003_037_0234_C_N_F_0,17,0.3679995369146758,0.1218442288592682,-0.695588613187432,-0.12387553593485646
1,C000003_037_0234_C_N_F_0,18,0.35130208333333335,0.1800925925925926,-0.6369791666666667,-0.14735577090395538
2,C000003_037_0234_C_N_F_0,18,0.3681843996153487,0.2235222980417327,-0.661547867435969,-0.19369614465727547


In [27]:
json_url_list = ['/Users/bumsoojoe/Desktop/팀프로젝트2_교통법규/120.개인형 이동장치 안전 데이터/01.데이터/2.Validation/라벨링데이터/VL1/CCTV/야간/맑음/연출',
'/Users/bumsoojoe/Desktop/팀프로젝트2_교통법규/120.개인형 이동장치 안전 데이터/01.데이터/2.Validation/라벨링데이터/VL1/CCTV/야간/우천/연출',
'/Users/bumsoojoe/Desktop/팀프로젝트2_교통법규/120.개인형 이동장치 안전 데이터/01.데이터/2.Validation/라벨링데이터/VL1/CCTV/주간/맑음/연출',
'/Users/bumsoojoe/Desktop/팀프로젝트2_교통법규/120.개인형 이동장치 안전 데이터/01.데이터/2.Validation/라벨링데이터/VL1/CCTV/주간/우천/연출'
]
img_url_list = ['/Users/bumsoojoe/Desktop/팀프로젝트2_교통법규/120.개인형 이동장치 안전 데이터/01.데이터/2.Validation/원천데이터/VS1/CCTV/야간/맑음/연출',
'/Users/bumsoojoe/Desktop/팀프로젝트2_교통법규/120.개인형 이동장치 안전 데이터/01.데이터/2.Validation/원천데이터/VS1/CCTV/야간/우천/연출',
'/Users/bumsoojoe/Desktop/팀프로젝트2_교통법규/120.개인형 이동장치 안전 데이터/01.데이터/2.Validation/원천데이터/VS1/CCTV/주간/맑음/연출',
'/Users/bumsoojoe/Desktop/팀프로젝트2_교통법규/120.개인형 이동장치 안전 데이터/01.데이터/2.Validation/원천데이터/VS1/CCTV/주간/우천/연출']

In [23]:
json = '/Users/bumsoojoe/Desktop/팀프로젝트2_교통법규/120.개인형 이동장치 안전 데이터/01.데이터/2.Validation/라벨링데이터/VL1'
img = '/Users/bumsoojoe/Desktop/팀프로젝트2_교통법규/120.개인형 이동장치 안전 데이터/01.데이터/2.Validation/원천데이터/VS1'


In [28]:
import os
import shutil

def move_files(source_folder, destination_folder):
    # 소스 디렉터리의 모든 파일 목록을 가져옵니다.
    files = [f for f in os.listdir(source_folder) if os.path.isfile(os.path.join(source_folder, f))]
    
    # 각 파일을 대상 디렉터리로 이동합니다.
    for file in files:
        shutil.move(os.path.join(source_folder, file), destination_folder)

# source_dir = 'A'  # 이동할 파일들이 있는 디렉터리
# dest_dir = 'B'    # 파일들을 이동시킬 대상 디렉터리

# move_files(source_dir, dest_dir)

dest_dir = '/Users/bumsoojoe/Desktop/팀프로젝트2_교통법규/120.개인형 이동장치 안전 데이터/01.데이터/2.Validation/라벨링데이터/VL1'
for source_dir in json_url_list:
    move_files(source_dir, dest_dir)

dest_dir =  '/Users/bumsoojoe/Desktop/팀프로젝트2_교통법규/120.개인형 이동장치 안전 데이터/01.데이터/2.Validation/원천데이터/VS1'
for source_dir in img_url_list:
    move_files(source_dir, dest_dir)